In [ ]:
import sys, os
repodir = os.path.abspath('..')
if not repodir in sys.path:
    sys.path.append(repodir)

In [ ]:
import itertools
import math

import numpy as np

from aircleaning import svg, html, produce

productsdir = produce.productsdir

In [ ]:
canvas = svg.draw_scene()
canvas

Natural ventilation source
https://www.ncbi.nlm.nih.gov/books/NBK143285/

Break room https://www.coronavirus.vic.gov.au/sites/default/files/2022-02/Break-and-lunch-room-ventilation-guide.pdf

Gym https://www.coronavirus.vic.gov.au/sites/default/files/2022-02/Gym-ventilation-guide.pdf

Function room https://www.coronavirus.vic.gov.au/sites/default/files/2022-02/Function-space-ventilation-guide.pdf

Exercise respiration https://www.pnas.org/doi/10.1073/pnas.2202521119

Energy Star

Mean wind speed: 3m/s in Melbourne (BoM)

In [ ]:
# produce.decision_tool(soft=True)

In [ ]:
soft = True

dim_range = range(3, 7, 1)
window_range = range(6)
door_range = range(3)
mech_range = range(2)
person_range = range(1, 6, 1)
activity_range = range(3)
allvols = tuple(sorted(set(map(np.product, itertools.combinations_with_replacement(dim_range, 3)))))
ach_range = range(0, 16)
cadrs = tuple(sorted(set(
    math.ceil(val / 100) * 100
    for val in map(np.product, tuple(itertools.product(ach_range, allvols)))))
    )

if not soft:
    outpath = os.path.join(productsdir, 'rooms')
    room_combos = tuple(itertools.product(
        dim_range, dim_range, dim_range,
        person_range, activity_range,
        window_range, door_range, mech_range,
        ))
    for room_combo in room_combos:
        scale = np.power(1/np.product(room_combo[:3]), 1/3) / np.power(1/np.min(allvols), 1/3)
        room = svg.draw_scene(*room_combo, size=2)
        room.projection.scale(scale)
        room.save_svg('_'.join(map(str, room_combo)), outpath)

if not soft:
    outpath = os.path.join(productsdir, 'costs')
    for cadr in cadrs:
        cost_analysis_by_cadr(max(100, cadr), path=outpath, name=str(cadr))

In [ ]:
10 * 3600 / 1000

In [ ]:
all_style = html.Style(
    # '''.container {''',
    # '''  max-width: 940px;''',
    # '''  margin: 0 auto;''',
    # '''  display: grid;''',
    # '''  grid-template-columns: 1fr 3fr;''',
    # '''  grid-gap: 10px;''',
    # '''}''',
    )

room_fields = html.Fieldset(
    'How big is your room?',
    {
        'Length (m):': html.CapturedInput(
            'range', 5, input_kwargs=dict(
                min=dim_range.start, max=dim_range.stop-1, step=dim_range.step,
                style="width: 90%;", name='room_length',
                ),
            ),
        'Width (m):': html.CapturedInput(
            'range', 5, input_kwargs=dict(
                min=dim_range.start, max=dim_range.stop-1, step=dim_range.step,
                style="width: 90%;", name='room_width',
                )
            ),
        'Height (m):': html.CapturedInput(
            'range', 3, input_kwargs=dict(
                min=dim_range.start, max=dim_range.stop-1, step=dim_range.step,
                style="width: 90%;", name='room_height',
                ),
            ),
        },
    html.Div(
        html.Div(
            "Total volume (m<sup>3</sup>):",
            style="float: left; width:70%; margin-right:10px",
            ),
        html.Div('-', identity='total_volume', style="float: left; width:20%"),
        style="margin-upper:10px"
        ),
    name='room_size',
    title="Dimensions",
    # style="margin: 10px;",
    )

usage_fields = html.Fieldset(
    "How is the room being used?",
    {
        'Number of people:': html.CapturedInput(
            'range', 1, input_kwargs=dict(
                min=person_range.start, max=person_range.stop-1, step=person_range.step,
                style="width: 90%;", name='number_people',
                )
            ),
        'Level of activity:': html.Div(
            html.Div(*(
                html.LabelledInput(
                    level,
                    html.Input('radio', i, checked=i==0, name='activity_level'),
                    )
                for i, level in enumerate(('Relaxed', 'Moderate', 'Intense'))
                ), style="display:flex;justify-content:center;align-items:center;"),
            style="margin:10px",
            ),
        },
    html.Div(
        html.Div(
            "Required cleaning (ACH):",
            style="float: left; width:70%; margin-right:10px",
            ),
        html.Div('-', identity='required_cleaning', style="float: left; width:20%"),
        style="margin-upper:10px"
        ),
    name='room_usage',
    title="Usage",
    # style="margin: 10px;",
    )

cleaning_fields = html.Fieldset(
    "How much cleaning is already available?",
    {
        'Number of open windows:': html.CapturedInput(
            'range', 0, input_kwargs=dict(
                min=window_range.start, max=window_range.stop-1, step=window_range.step,
                style="width: 90%;", name='number_windows',
                ),
            ),
        'Number of open doors:': html.CapturedInput(
            'range', 0, input_kwargs=dict(
                min=door_range.start, max=door_range.stop-1, step=door_range.step,
                style="width: 90%;", name='number_doors',
                ),
            ),
        'Mechanical ventilation:': html.LabelledInput(
            'Yes', html.Input('checkbox', 0, name='mech_vent'),
            ),
        },
    html.Div(
        html.Div(
            "Provided cleaning (ACH):",
            style="float: left; width:70%; margin-right:10px",
            ),
        html.Div('-', identity='provided_cleaning', style="float: left; width:20%"),
        style="margin-upper:10px"
        ),
    name='room_cleaning',
    title="Circulation",
    # style="margin: 10px;",
    )

room_viz = html.Div(
    html.Image(
        "https://via.placeholder.com/150", identity="room_viz",
        style="max-width:100%; max-height:100%;"
        ),
    # style="float: right; width: 50%; margin: 10px;",
    )

input_selector = html.TabbedPanes(
    room_fields,
    usage_fields,
    cleaning_fields,
    identity="input_selector",
    # style="width: 45%; float: left; text-align:center;",
    )

input_form = html.Form(
    input_selector,
    identity='user_form',
    )

input_section = html.Div(
    input_form,
    room_viz,
    style="display: grid; grid-template-columns: 60% 40%; overflow:auto"
    # style="width:100%;"
    # style="display:flex;justify-content:center; margin:10px; width:100%",
    # style="display:flex;justify-content:center;align-items:center; margin:10px",
    )

summary_widget = html.Div(
    html.Div(
        "<b>Extra cleaning required:</b>",
        style="display:flex;justify-items:center;"
        ),
    html.Div(
        html.Div(
            html.Div(
                "<i>Air changes (ACH):</i>",
                ),
            html.Div(
                '-',
                identity='extra_cleaning',
                ),
            style=(
                '''display: grid;'''
                '''grid-template-columns: 1fr 1fr;'''
                '''justify-items:center;'''
                ),
            ),
        html.Div(
            html.Div(
                "<i>CADR (m<sup>3</sup>/hr):</i>",
                ),
            html.Div(
                '-',
                identity='extra_cleaning_cadr',
                ),
            style=(
                '''display: grid;'''
                '''grid-template-columns: 1fr 1fr;'''
                '''justify-items:center;'''
                ),
            ),
        style=(
            '''display: grid;'''
            '''grid-template-rows: 1fr 1fr;'''
            ),
        ),
    style=(
        '''display: grid;'''
        '''grid-template-rows: 30px 1fr;'''
        '''justify-items: center;'''
        ),
    )

summary_section = html.Div(
    summary_widget,
    )

tool_selector = html.TabbedPanes(
    html.Image(
        '-',
        identity="air_cleaner_recommendations",
        style="display:flex;justify-content:center;align-items:center; margin:10px",
        title="Air cleaners",
        ),
    # html.Image(
    #     "https://via.placeholder.com/150",
    #     style="display:flex;justify-content:center;align-items:center; margin:10px",
    #     title="Air quality",
    #     ),
    # html.Image(
    #     "https://via.placeholder.com/150",
    #     style="display:flex;justify-content:center;align-items:center; margin:10px",
    #     title="Infection risk",
    #     ),
    identity="tool_selector",
    # style="width: 100%",
    )

output_section = html.Div(
    tool_selector,
    style="display: grid; grid-template-rows: 1fr 3fr;",
    )

all_content = html.Div(
    input_section,
    summary_section,
    output_section,
    style=(
        '''display: grid;'''
        '''grid-template-rows: 25% 7% 68%;'''
        '''max-height: 11.7in;'''
        '''max-width: 8.3in;'''
        )
    # classes=('container',)
    )

science_code = html.Script(
    '''function calculate_natural_vent_rate(windows, doors){''',
    '''  const wind_speed = 3;''',
    '''  const window_area = 0.5 * 0.8 * windows;''',
    '''  const door_area = 0.6 * 2 * doors;''',
    '''  if (windows>0 && doors>0) {''',
    '''    var coeff = 0.1;''',
    '''    var aperture = Math.min(window_area, door_area);'''
    '''  } else {''',
    '''    var coeff = 0.05;''',
    '''    var aperture = Math.max(window_area, door_area);'''
    '''  };''',
    '''  return coeff * wind_speed * aperture * 3600;''',
    '''};''',
    '''function calculate_mech_vent_rate(mech, volume){''',
    '''  return mech * 6 * volume;''',
    '''};''',
    '''function calculate_vent_rate(windows, doors, mech, volume){''',
    '''  const natural = calculate_natural_vent_rate(windows, doors);''',
    '''  const mechanical = calculate_mech_vent_rate(mech, volume);''',
    '''  const baseline = 3 * volume;'''
    '''  return natural + mechanical + baseline'''
    '''};''',
    '''function calculate_fouling_rate(persons, activity){''',
    '''  const l_s_pp = 10;'''
    '''  const m3_h_pp = l_s_pp * 3600 / 1000;'''
    '''  const multiplier = [1, 3, 10][activity];'''
    '''  return multiplier * m3_h_pp * persons;''',
    '''};''',
    )

form_update = html.Script(
    '''function form_update(form){''',
    '''    const productsdir = "https://rsbyrne.github.io/aircleaning/products";'''
    '''    const length = form.room_length.value;''',
    '''    const width = form.room_width.value;''',
    '''    const height = form.room_height.value;''',
    '''    const windows = form.number_windows.value;''',
    '''    const doors = form.number_doors.value;''',
    '''    const persons = form.number_people.value;''',
    '''    const room = document.getElementById('room_viz');''',
    '''    const mech = form.mech_vent.value;'''
    '''    const activity_level = parseInt(form.activity_level.value);''',
    '''    room.src =''',
    '''        productsdir + "/rooms/" +''',
    '''        [length, width, height, persons, activity_level, windows, doors, mech].join("_")''',
    '''        + ".svg";''',
    '''    const volume = form.room_length.value * form.room_width.value * form.room_height.value;''',
    '''    document.getElementById("total_volume").innerHTML = volume;''',
    '''    const natural_vent = calculate_natural_vent_rate(windows, doors);''',
    '''    const mech_vent = calculate_mech_vent_rate(mech, volume);''',
    '''    const provided_cleaning = Math.round((natural_vent + mech_vent) / volume);''',
    '''    document.getElementById("provided_cleaning").innerHTML = provided_cleaning;''',
    '''    const required_cleaning = calculate_fouling_rate(persons, activity_level);'''
    '''    document.getElementById("required_cleaning").innerHTML = required_cleaning;'''
    '''    const extra_cleaning = Math.max(0, required_cleaning - provided_cleaning);'''
    '''    document.getElementById("extra_cleaning").innerHTML = extra_cleaning;''',
    '''    const extra_cleaning_cadr = Math.ceil(extra_cleaning * volume / 100) * 100;'''
    '''    document.getElementById("extra_cleaning_cadr").innerHTML = extra_cleaning_cadr;''',
    '''    const acchart = document.getElementById("air_cleaner_recommendations");''',
    '''    acchart.src =''',
    '''        productsdir + "/costs/" +''',
    '''        Math.max(100, extra_cleaning_cadr)''',
    '''        + ".png";''',
    '''}''',
    )

initialise_script = html.Script(
    '''form_update(document.getElementById("user_form"));''',
    '''document.getElementById("tool_selector_button_0").click();'''
    '''document.getElementById("input_selector_button_0").click();'''
    )

page = html.Page(
    science_code,
    form_update,
    all_style,
    html.Div(
        all_content,
        style=(
            '''display: flex;'''
            '''justify-items: center;'''
            '''flex-wrap: wrap;'''
            '''align-items: center;'''
            '''justify-content: center;'''
            )
        ),
    initialise_script,
    # style="float:center;margin:auto",
    )

display(page)
page.save_html('sample')

In [ ]:
html.Image("http://localhost:8889/lab/tree/products/rooms/3_3_3_1_1.svg").save_html('test')

In [ ]:
html.Image('https://rsbyrne.github.io/aircleaning/products/0_0.png')

In [ ]:
allvols

ach in range(0, 16)